In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
# import torchvision
from torch.utils.tensorboard import SummaryWriter

In [ ]:
!ls

In [ ]:
writer = SummaryWriter("/home/jupyter/BengaliAi/logs")

In [ ]:
# !pip install tb-nightly

In [ ]:
# !pip install future

In [ ]:
# !pip install albumentations==0.0.10

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
def compute():
    df = pd.read_csv("input/train.csv")
    print(df.head())
    df.loc[:,"kfold"] = -1
    
    df = df.sample(frac=1).reset_index(drop=True)
    
    X = df.image_id.values
    y = df[["grapheme_root", "vowel_diacritic", "consonant_diacritic"]].values
    
    mskf = MultilabelStratifiedKFold(n_splits=5)
# #     for trn_,val_ in mskf.split(X,y):
# #         print("TRAIN:", trn_, "Val:", val_)
    for fold, (trn_,val_) in enumerate(mskf.split(X,y)):
        print("TRAIN:", len(trn_), "Val:", len(val_))
# #         df.loc[val_,"kfold"] = fold

# #     print(df.kfold.value_counts())
# #     df.to_csv("input/train_folds.csv", index=False)
compute()

In [ ]:
import numpy as np
import joblib
import glob
from tqdm import tqdm
from PIL import Image
import albumentations
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def create_image_pickels():
    files = glob.glob("input/train_image_data_*.parquet")
    for f in files:
        df = pd.read_parquet(f)
        image_ids = df.image_id.values
        df = df.drop("image_id",axis=1)
        image_array = df.values
        for j,img_id in tqdm(enumerate(image_ids),total=len(image_ids)):
            joblib.dump(image_array[j,:],f"input/image_pickels/{img_id}.pkl")
create_image_pickels()

In [ ]:
class BengaliDatasetTrain:
    def __init__(self,folds,img_height,img_width,mean,std):
        df = pd.read_csv("input/train_folds.csv")
        df = df.drop("grapheme",axis=1)
#         df = [["image_id","grapheme_root", "vowel_diacritic", "consonant_diacritic", "kfold"]]
        q = df.kfold.isin(folds)
        df = df[q].reset_index(drop=True)
        self.image_ids = df.image_id.values
        self.grapheme_root = df.grapheme_root.values
        self.vowel_diacritic = df.vowel_diacritic.values
        self.consonant_diacritic = df.consonant_diacritic.values
        
        if(len(folds)==1):
            self.aug = albumentations.Compose([
                albumentations.Resize(img_height,img_width,always_apply=True),
                albumentations.Normalize(mean,std,always_apply=True)
            ])
        else:
            self.aug = albumentations.Compose([
                albumentations.Resize(img_height,img_width,always_apply=True),
                albumentations.ShiftScaleRotate(shift_limit=0.0625,
                                                scale_limit=0.1,
                                                rotate_limit=5,
                                                p=0.9),
                albumentations.Normalize(mean,std,always_apply=True)
            ])
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self,item):
#         print(self.image_ids[item])
        image = joblib.load(f"input/image_pickels/{self.image_ids[item]}.pkl")
        image = image.reshape(137,236).astype(float)
        image = Image.fromarray(image).convert("RGB")
        
        image = self.aug(image = np.array(image))["image"]
        image = np.transpose(image,(2,0,1)).astype(np.float32)
        return{
            'image': torch.tensor(image,dtype=torch.float),
            'grapheme_root': torch.tensor(self.grapheme_root[item],dtype=torch.long),
            'vowel_diacritic': torch.tensor(self.vowel_diacritic[item],dtype=torch.long),
            'consonant_diacritic': torch.tensor(self.consonant_diacritic[item],dtype=torch.long)
        }
       
        
        
        
        
        
        

In [ ]:
datset = BengaliDatasetTrain(folds=[0,1],img_height=137, img_width=236,mean=(0.485,0.456,0.406), std=(0.229, 0.224, 0.225))

In [ ]:
len(datset)

In [ ]:
idx=1
img = datset[idx]["image"]
print(datset[idx]["grapheme_root"])
print(datset[idx]["vowel_diacritic"])
print(datset[idx]["consonant_diacritic"])
npimg = img.numpy()
plt.imshow(np.transpose(npimg,(1,2,0)))

In [ ]:
import pretrainedmodels
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
class ResNet34(nn.Module):
    def __init__(self,pretrained):
        super(ResNet34,self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained="imagenet")
        else:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained=None)
        
        self.l0 = nn.Linear(512,168)
        self.l1 = nn.Linear(512,11)
        self.l2 = nn.Linear(512,7)
    
    def forward(self,x):
        bs,_,_,_ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x,1).reshape(bs,-1)
        l0 = self.l0(x)
        l1 = self.l1(x)
        l2 = self.l2(x)
        
        return l0,l1,l2
        
        
    

In [ ]:
MODEL_DISPATCHER = {
    'resnet34': ResNet34
}

In [ ]:
model = MODEL_DISPATCHER["resnet34"](pretrained=False)
# model

In [ ]:
import os
import ast

In [ ]:
# !sh src/run.sh

In [ ]:
DEVICE = "cuda"
# TRAINING_FOLDS_CSV = os.getenv("TRAINING_FOLDS_CSV")
# IMG_HEIGHT = os.environ.get("IMG_HEIGHT")
# IMG_WIDTH = os.environ.get("IMG_WIDTH")
# EPOCHS = os.environ.get("EPOCHS")

# TRAIN_BATCH_SIZE = os.environ.get("TRAIN_BATCH_SIZE")
# TEST_BATCH_SIZE = os.environ.get("TEST_BATCH_SIZE")

# MODEL_MEAN = os.environ.get("MODEL_MEAN")
# MODEL_STD = os.environ.get("MODEL_STD")

# TRAINING_FOLDS = os.environ.get("TRAINING_FOLDS")
# VALIDATION_FOLDS =os.environ.get("VALIDATION_FOLDS")
# BASE_MODEL = os.environ.get("BASE_MODEL")
CUDA_VISIBLE_DEVICES=0
IMG_HEIGHT=137
IMG_WIDTH=236
EPCOHS=10
TRAIN_BATCH_SIZE=256
TEST_BATCH_SIZE=8
MODEL_MEAN=(0.485,0.456,0.406)
MODEL_STD=(0.229, 0.224, 0.225)
BASE_MODEL="resnet34"

TRAINING_FOLDS=(0,1,2,3)
VALIDATION_FOLDS=(4,)

In [ ]:
print(VALIDATION_FOLDS)

In [ ]:
model = MODEL_DISPATCHER[BASE_MODEL](pretrained=True)
model.to(DEVICE)

train_dataset = BengaliDatasetTrain(
    folds=TRAINING_FOLDS,
    img_height=IMG_HEIGHT,
    img_width=IMG_WIDTH,
    mean=MODEL_MEAN,
    std=MODEL_STD
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

valid_dataset = BengaliDatasetTrain(
    folds=VALIDATION_FOLDS,
    img_height=IMG_HEIGHT,
    img_width=IMG_WIDTH,
    mean=MODEL_MEAN,
    std=MODEL_STD
)

valid_loader = torch.utils.data.DataLoader(
    dataset=valid_dataset,
    batch_size=TEST_BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode="min",patience=5,factor=0.3,verbose=True)

if torch.cuda.device_count()>1:
    model = nn.DataParallel()



In [ ]:
def loss_fn(outputs,targets):
    o1,o2,o3 = outputs
    t1,t2,t3 = targets
    
    l1 = nn.CrossEntropyLoss()(o1,t1)
    l2 = nn.CrossEntropyLoss()(o2,t2)
    l3 = nn.CrossEntropyLoss()(o3,t3)
    return (l1+l2+l3) / 3
    
    
def train(dataset,data_loader, model, optimizer,epoch):
    model.train()
    
    for bi, d in tqdm(enumerate(data_loader), total=int((len(dataset)/data_loader.batch_size))):
        image = d["image"]
        grapheme_root = d["grapheme_root"]
        vowel_diacritic = d["vowel_diacritic"]
        consonant_diacritic = d["consonant_diacritic"]
        
        image = image.to(DEVICE,dtype=torch.float)
        grapheme_root = grapheme_root.to(DEVICE,dtype=torch.long)
        vowel_diacritic = vowel_diacritic.to(DEVICE,dtype=torch.long)
        consonant_diacritic = consonant_diacritic.to(DEVICE,dtype=torch.long)
        
        optimizer.zero_grad()
        outputs = model(image)
        targets = (grapheme_root,vowel_diacritic,consonant_diacritic)
        loss = loss_fn(outputs,targets)
        
        loss.backward()
        optimizer.step()
        if bi % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, bi * len(d), len(data_loader.dataset),
                       100. * bi / len(data_loader), loss.item()))
    # Record loss into the writer
    writer.add_scalar('Train/Loss', loss.item(), epoch)
    writer.flush()
        
def evaluate(dataset,data_loader, model):
    model.eval()
    final_loss=0
    counter=0
    for bi, d in tqdm(enumerate(data_loader), total=int((len(dataset)/data_loader.batch_size))):
        counter = counter+1
        image = d["image"]
        grapheme_root = d["grapheme_root"]
        vowel_diacritic = d["vowel_diacritic"]
        consonant_diacritic = d["consonant_diacritic"]
        
        image = image.to(DEVICE,dtype=torch.float)
        grapheme_root = grapheme_root.to(DEVICE,dtype=torch.long)
        vowel_diacritic = vowel_diacritic.to(DEVICE,dtype=torch.long)
        consonant_diacritic = consonant_diacritic.to(DEVICE,dtype=torch.long)
        
        outputs = model(image)
        targets = (grapheme_root,vowel_diacritic,consonant_diacritic)
        loss = loss_fn(outputs,targets)
        final_loss +=loss
    return final_loss / counter

        
    

In [ ]:
for epoch in range(EPCOHS):
    train(train_dataset,train_loader,model,optimizer,epoch)
    with torch.no_grad():
        val_score = evaluate(valid_dataset,valid_loader,model)
        scheduler.step(val_score)
    torch.save(model.state_dict(),f"{BASE_MODEL}_fold{VALIDATION_FOLDS[0]}.bin")

In [ ]:
writer.close()

In [ ]:
TRAINING_FOLDS=(0,1,2,4)
VALIDATION_FOLDS=(3,)

for epoch in range(EPCOHS):
    train(train_dataset,train_loader,model,optimizer,epoch)
    with torch.no_grad():
        val_score = evaluate(valid_dataset,valid_loader,model)
        scheduler.step(val_score)
    torch.save(model.state_dict(),f"{BASE_MODEL}_fold{VALIDATION_FOLDS[0]}.bin")

In [ ]:
TRAINING_FOLDS=(0,1,4,3)
VALIDATION_FOLDS=(2,)

for epoch in range(EPCOHS):
    train(train_dataset,train_loader,model,optimizer)
    val_score = evaluate(valid_dataset,valid_loader,model)
    scheduler.step(val_score)
    torch.save(model.state_dict(),f"{BASE_MODEL}_fold{VALIDATION_FOLDS[0]}.bin")

In [ ]:
TRAINING_FOLDS=(0,4,2,3)
VALIDATION_FOLDS=(1,)

for epoch in range(EPCOHS):
    train(train_dataset,train_loader,model,optimizer)
    val_score = evaluate(valid_dataset,valid_loader,model)
    scheduler.step(val_score)
    torch.save(model.state_dict(),f"{BASE_MODEL}_fold{VALIDATION_FOLDS[0]}.bin")

In [ ]:
TRAINING_FOLDS=(4,1,2,3)
VALIDATION_FOLDS=(0,)

for epoch in range(EPCOHS):
    train(train_dataset,train_loader,model,optimizer)
    val_score = evaluate(valid_dataset,valid_loader,model)
    scheduler.step(val_score)
    torch.save(model.state_dict(),f"{BASE_MODEL}_fold{VALIDATION_FOLDS[0]}.bin")